# data preparation for delta_roi+upfront discount + agreement month according to sakshi's data

# after deviation data correction using conversion data

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import sys
sys.path.insert(1, 'C:\\Users\\user\\Desktop\\Data\\conversion_project\\codes')
from featexp import get_univariate_plots,get_trend_stats
%matplotlib inline
from sklearn.model_selection import train_test_split
import xgboost as xgb 
from numpy import NAN
import math
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [2]:
data5=pd.read_feather("../../base_data_oot_jan_24_05_2022.feather")
data5['APP_AQB']=np.where(data5['COBO_EXEC_DEF_AQB'].isnull() , data5['PRIMARY_NET_AQB'], (data5['PRIMARY_NET_AQB'] +data5['COBO_EXEC_DEF_AQB'])    )
data5=data5[(data5['CHANNEL_MAIN']=='C2C') & (data5['SUB_CHANNEL']!='Dealer')]
data5=data5[data5['agg_flag2']==1]
data5['PRIMARY_EMP_TYPE']=data5['PRIMARY_EMP_TYPE'].str.lower()
data5['SUB_CHANNEL'].value_counts()

C2C GS       11798
C2C BB         439
C24 Store       12
Name: SUB_CHANNEL, dtype: int64

# merged with AJ DATA

In [3]:
data5['AGREEMENT_DATE']=data5['AGRE_LATEST_TIMESTAMP']
data5['AGREEMENT_MONTH']=data5['AGREEMENT_DATE'].astype(str).str[0:7]
data5['AGREEMENT_day']=data5['AGREEMENT_DATE'].astype(str).str[8:11]
data5['AGREEMENT_day']=pd.to_numeric(data5['AGREEMENT_day'])
data5['AGREEMENT_day'].value_counts()
data5=data5[data5['AGREEMENT_MONTH']>'2021-12']
data5[['FINAL_PC_ROI_AJ','AGREEMENT_MONTH','DISBURSED_ROI']].groupby(['AGREEMENT_MONTH']).apply(lambda x: x.isnull().sum()*100/len(x)).T

AGREEMENT_MONTH,2022-01,2022-02,2022-03,2022-04,2022-05
FINAL_PC_ROI_AJ,2.613065,0.0,0.0,0.0,0.262697
AGREEMENT_MONTH,0.000000,0.0,0.0,0.0,0.000000
DISBURSED_ROI,0.000000,0.0,0.0,0.0,0.000000


In [4]:
data5['AGREEMENT_MONTH'].value_counts()

2022-04    1301
2022-03    1284
2022-05    1142
2022-02    1095
2022-01     995
Name: AGREEMENT_MONTH, dtype: int64

In [5]:
data5.shape

(5817, 92)

In [6]:
data5=data5[~data5['FINAL_PC_RB_V4_AJ'].isnull()]
data5.shape

(5811, 92)

In [7]:
data5['AGREEMENT_MONTH'].value_counts()

2022-04    1301
2022-03    1284
2022-05    1142
2022-02    1095
2022-01     989
Name: AGREEMENT_MONTH, dtype: int64

In [8]:
def roundfun(x, base=0.0025):
    if type(x) == str:
        return x
    elif math.isnan(x):
        return np.nan #.astype('str')
    else:
        return base * round(x/base)
data5['DISBURSED_ROI']=data5['DISBURSED_ROI'].apply(roundfun)

In [ ]:
1:{'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
2:{'A':14,'B':15,'C':16,'D':16.5,'E':19.75},
3:{'A':14,'B':15,'C':16,'D':16.5,'E':19.75},
4:{'A':14,'B':15,'C':16,'D':17, 'E':19.75}}
5:{'A':14,'B':15,'C':16,'D':17, 'E':19.75}}


In [9]:
#ds roi 
replace_percent={9: {'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
 10:{'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
11:{'A':14,'B':15,'C':16,'D':17,'E':21},
12:{'A':14,'B':15,'C':16,'D':17,'E':21},
1:{'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
2:{'A':14,'B':15,'C':16,'D':16.5,'E':19.75},
4:{'A':14,'B':15,'C':16,'D':17, 'E':19.75}}

def rb_bin_assign(row):
#     print(row['FINAL_PC_RB_V4_AJ'])
    if row['AGREEMENT_MONTH']=='2021-09':
        return replace_percent[9][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-10':
        return replace_percent[10][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-11':
        return replace_percent[11][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-12':
        return replace_percent[12][row['FINAL_PC_RB_V4_AJ']]
    elif ((row['AGREEMENT_MONTH']=='2021-12') & (row['AGREEMENT_day']<=3)):
        return replace_percent[12][row['FINAL_PC_RB_V4_AJ']]
    elif ((row['AGREEMENT_MONTH']=='2022-01') & (row['AGREEMENT_day']>3)):
        return replace_percent[1][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-02' :
        return replace_percent[2][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-03' :
        return replace_percent[2][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-04' :
        return replace_percent[4][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-05' :
        return replace_percent[4][row['FINAL_PC_RB_V4_AJ']]
    else:
        return 0
data5['new_DS_ROI']=data5.apply(rb_bin_assign, axis=1)
data5['new_DS_ROI']=data5['new_DS_ROI']/100
data5['delta_roi']=data5['DISBURSED_ROI']-data5['new_DS_ROI']
data5['delta_roi']=data5['delta_roi']*100

In [10]:
#keep only recent 3 months of data
data_df=data5.copy()
data_df=data_df[~data_df['delta_roi'].isnull()]
data_df.groupby(['AGREEMENT_MONTH']).size()

AGREEMENT_MONTH
2022-01     989
2022-02    1095
2022-03    1284
2022-04    1301
2022-05    1142
dtype: int64

In [11]:
# data_df['delta_roi_bin'] = pd.cut(data_df.delta_roi, [-np.inf,-0.5,0.5,+np.inf])
def bin_assign(a):
    if a<-0.25:
        return 0
    elif a>0.25:
        return 2
    else:
        return 1
data_df['delta_roi_bin_binary']=data_df.apply(lambda x: bin_assign(x['delta_roi']), axis=1)
data_df['delta_roi_bin_binary'].value_counts(dropna=False,sort=True)

2    2607
1    1616
0    1588
Name: delta_roi_bin_binary, dtype: int64

In [12]:
data_df['delta_roi_bin_binary'].replace({0:'Negative Deviation',2:'Positive Deviation',1:'No Deviation'}).value_counts(normalize=True,sort=False)*100

Negative Deviation    27.327482
Positive Deviation    44.863191
No Deviation          27.809327
Name: delta_roi_bin_binary, dtype: float64

# change delta roi based on BI rules

In [13]:
city_state_master=pd.read_csv("states_pincodes.csv")
def get_region_from_pincode(pincode):
    try:
        region = city_state_master.loc[city_state_master["pincode"] == int(pincode)]["region"].to_list()[0]
    except:
        region = 'North'
    return region

data_df['region_pincode']=data_df['PINCODE '].apply(get_region_from_pincode)
data_df['region_pincode'].value_counts(dropna=False)

North    2077
South    1836
West     1721
East      177
Name: region_pincode, dtype: int64

In [14]:
policy1_df=pd.read_csv("aqb_by_emi_lt_0.5.csv")
policy1_df

,AQB/EMI<0.5,A,B,C,D,E
0,Cibil <700,0.145,0.155,0.165,0.185,0.24
1,Cibil 700-750,0.140,0.150,0.160,0.180,0.23
2,Cibil- 750-765,0.140,0.150,0.160,0.180,0.22
3,Cibil- 765-785,0.140,0.150,0.160,0.180,0.22
4,Cibil- 785+,0.140,0.150,0.160,0.180,0.22
5,NTC,0.140,0.150,0.160,0.180,0.23


In [15]:
policy_sal=pd.read_csv("aqb_by_emi_gt_0.5_sal.csv")
policy_sal

,AQB/EMI>0.5,A,B,C,D,E,Unnamed: 6
0,Cibil <700,0.145,0.155,0.165,0.18,0.2100,NaN
1,Cibil 700-750,0.140,0.150,0.160,0.17,0.1975,NaN
2,Cibil- 750-765,0.130,0.150,0.160,0.17,0.1975,NaN
3,Cibil- 765-785,0.120,0.140,0.150,0.17,0.1975,NaN
4,Cibil- 785+,0.120,0.130,0.140,0.17,0.1975,NaN
5,NTC,0.140,0.150,0.160,0.17,0.1975,NaN


In [16]:
policy_self=pd.read_csv("aqb_by_emi_gt_0.5_self.csv")
policy_self

,AQB/EMI>0.5,A,B,C,D,E
0,Cibil <700,0.145,0.155,0.165,0.18,0.2100
1,Cibil 700-750,0.140,0.150,0.160,0.17,0.1975
2,Cibil- 750-765,0.140,0.150,0.160,0.17,0.1975
3,Cibil- 765-785,0.140,0.150,0.160,0.17,0.1975
4,Cibil- 785+,0.125,0.140,0.150,0.17,0.1975
5,NTC,0.140,0.150,0.160,0.17,0.1975


In [17]:
def feb_24_dev_march(row,cibil_col_name='CIBIL_MASTER_DATA2',rb_col_name='FINAL_PC_RB_V4_AJ',app_inc_col_name='INCOME_MASTER_DATA2'):

    bi_dev=0
    #first overaching rules
    print(row['INCOME_MASTER_DATA2'])  
#     print(row['PRIMARY_EMP_TYPE'])  
#     print(row[cibil_col_name],row['AQB_BY_EMI'],row[rb_col_name])
    
    if (((row['PRIMARY_EMP_TYPE']=="salaried")|(row['PRIMARY_EMP_TYPE']=="pensioner")) & (row[app_inc_col_name]<30000)):
        bi_dev+=0.005
        print("1")
    if row['PRIMARY_EMP_TYPE']=='self-employed':
        bi_dev+=0.005
    if row[cibil_col_name]==None:
        bi_dev+=-0.01
    elif row[cibil_col_name]<300:
        bi_dev+=-0.01
    print(bi_dev)
    if row['AQB_BY_EMI']<0.5:
        if (row[cibil_col_name]==None):
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][5]
        elif row[cibil_col_name]<300:
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][5]
        elif (row[cibil_col_name]>=300) & (row[cibil_col_name]<700) :
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][0]
        elif (row[cibil_col_name]>=700) & (row[cibil_col_name]<750) :
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][1]
        elif (row[cibil_col_name]>=750) & (row[cibil_col_name]<765) :
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][2]
        elif (row[cibil_col_name]>=765) & (row[cibil_col_name]<785) :
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][3]
        elif (row[cibil_col_name]>=785) :
               bi_dev+=policy1_df.loc[:,row[rb_col_name][0]][4]
        print(bi_dev,1)
                                                          
    elif row['AQB_BY_EMI']>=0.5:
        if ((row['PRIMARY_EMP_TYPE']=="salaried")|(row['PRIMARY_EMP_TYPE']=="pensioner")):
                    if (row[cibil_col_name]==None):
                          bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][5]
                    elif (row[cibil_col_name]<300) :
                          bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][5]
                    elif (row[cibil_col_name]>=300) & (row[cibil_col_name]<700) :
                           bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][0]
                    elif (row[cibil_col_name]>=700) & (row[cibil_col_name]<750) :
                           bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][1]
                    elif (row[cibil_col_name]>=750) & (row[cibil_col_name]<765) :
                           bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][2]
                    elif (row[cibil_col_name]>=765) & (row[cibil_col_name]<785) :
                           bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][3]
                    elif (row[cibil_col_name]>=785) :
                          bi_dev+=policy_sal.loc[:,row[rb_col_name][0]][4]
                    print(bi_dev,2)
                                      
        elif row['PRIMARY_EMP_TYPE']=='self-employed':
                    if (row[cibil_col_name]==None):
                          bi_dev+=policy_self.loc[:,row[rb_col_name][0]][5]
                    elif (row[cibil_col_name]<300) :
                          bi_dev+=policy_self.loc[:,row[rb_col_name][0]][5]
                    elif (row[cibil_col_name]>=300) & (row[cibil_col_name]<700) :
                           bi_dev+=policy_self.loc[:,row[rb_col_name][0]][0]
                    elif (row[cibil_col_name]>=700) & (row[cibil_col_name]<750) :
                           bi_dev+=policy_self.loc[:,row[rb_col_name][0]][1]
                    elif (row[cibil_col_name]>=750) & (row[cibil_col_name]<765) :
                           bi_dev+=policy_self.loc[:,row[rb_col_name][0]][2]
                    elif (row[cibil_col_name]>=765) & (row[cibil_col_name]<785) :
                           bi_dev+=policy_self.loc[:,row[rb_col_name][0]][3]
                    elif (row[cibil_col_name]>=785) :
                           bi_dev+=policy_self.loc[:,row[rb_col_name][0]][4] 
                    print(bi_dev,3)      
    return bi_dev    

l_id=700760
t1=data_df[data_df['LEAD_ID']==l_id]
t1.apply(feb_24_dev_march, axis=1)

35500.0
0
0.15 2


5845    0.15
dtype: float64

In [18]:
def jan_till_24_feb(row,cibil_col_name='CIBIL_MASTER_DATA2',rb_col_name='FINAL_PC_RB_V4_AJ',app_inc_col_name='INCOME_MASTER_DATA2'):
    '''
    Note : NTC rules are not explicitly mentioned in the rule book. but, we are excluding NTC cases from premium ROI.
    '''
#     rb_dict={'A':0.1402,'B':0.1504,'C':0.161,'D':0.17,'E':0.1775}
    rb_dict={'A':0.140,'B':0.150,'C':0.16,'D':0.17,'E':0.1975}
    
    #user curr city pincode
    target_roi = rb_dict[row[rb_col_name]]
    roi=target_roi
    max_premium_cap = 0.0050

    
    region = row['region_pincode']
    risk_bucket=row[rb_col_name]
    
    if region == None:
        roi = target_roi
    elif row[cibil_col_name]!=None:
        if region == "North" :
            print(row["PRIMARY_EMP_TYPE"] == "self-employed",row["PRIMARY_EMP_TYPE"])
            print((row["PRIMARY_EMP_TYPE"] == "salaried" and row[app_inc_col_name] < 30000),2)
            print((row[cibil_col_name] >= 300 and row[cibil_col_name] < 700),3)
            print(row[cibil_col_name],region,row[app_inc_col_name])
            if risk_bucket == "A" or risk_bucket == "B" or risk_bucket == "C":
                
                if (row["PRIMARY_EMP_TYPE"] == "self-employed") or (row["PRIMARY_EMP_TYPE"] == "salaried" and row[app_inc_col_name] < 30000) or (row[cibil_col_name] >= 300 and row[cibil_col_name] < 700):
                    roi = target_roi + 0.0050
#                     print("here")
                    
                    
                # if tier == "Tier-3":
                #     # add later
        elif region == "South":
            if risk_bucket == "A" or risk_bucket == "B" or risk_bucket == "C":
                if (row["PRIMARY_EMP_TYPE"] == "self-employed") or (row["PRIMARY_EMP_TYPE"] == "salaried" and row[app_inc_col_name] < 30000) or (row[cibil_col_name] >= 300 and row[cibil_col_name] < 700):
                    roi = target_roi + 0.0050
                # if tier == "Tier-3":
                #     # add later
        elif region == "West" or region == "East":
            if risk_bucket == "A" or risk_bucket == "B" or risk_bucket == "C":
                if row[cibil_col_name] >= 300 and row[cibil_col_name] < 700:
                    roi = target_roi + 0.0050
                # if tier == "Tier-3":
                #     # add later
        else:
            roi = target_roi

    # capping for maximum premium limit
    print(roi,target_roi)
    if roi > (target_roi + max_premium_cap):
#         print("inside")
        roi = target_roi + max_premium_cap
    return roi
# 652116
t1=data_df[data_df['LEAD_ID']==515319]
t1.apply(jan_till_24_feb, axis=1)

0.155 0.15


15474    0.155
dtype: float64

## split data based on last deployed change

In [19]:
data_df_jan=data_df[data_df['AGREEMENT_MONTH']<'2022-02']
data_df_feb_march=data_df[data_df['AGREEMENT_MONTH']>='2022-02']

In [20]:
old_tmp=data_df_feb_march[(data_df_feb_march['AGREEMENT_MONTH']=='2022-02')&(data_df_feb_march['AGREEMENT_day']<24) ]
data_df_feb_march=data_df_feb_march[~((data_df_feb_march['AGREEMENT_MONTH']=='2022-02')&(data_df_feb_march['AGREEMENT_day']<24)) ]

In [21]:
data_df_jan=pd.concat([data_df_jan,old_tmp],axis=0)

In [22]:
data_df_jan['DS_BI_ROI'] = data_df_jan.apply(jan_till_24_feb, axis=1)
data_df_feb_march['DS_BI_ROI'] = data_df_feb_march.apply(feb_24_dev_march, axis=1)

0.14 0.14
True self-employed
False 2
False 3
789.0 North 146971.42857142858
0.14500000000000002 0.14
0.14 0.14
False salaried
True 2
False 3
134.0 North 21180.0
0.17 0.17
0.15 0.15
True self-employed
False 2
False 3
723.0 North 40000.0
0.155 0.15
0.15 0.15
True self-employed
False 2
False 3
782.0 North 41599.0
0.165 0.16
0.15 0.15
False salaried
False 2
False 3
739.0 North 80000.0
0.16 0.16
False salaried
False 2
False 3
748.0 North 50000.0
0.16 0.16
True self-employed
False 2
False 3
704.0 North 29166.0
0.155 0.15
0.155 0.15
0.16 0.16
0.15 0.15
0.15 0.15
0.165 0.16
0.165 0.16
0.14 0.14
False salaried
False 2
False 3
701.0 North 52715.66666666666
0.16 0.16
0.16 0.16
0.14 0.14
0.16 0.16
0.16 0.16
0.155 0.15
0.14500000000000002 0.14
0.165 0.16
0.14 0.14
0.16 0.16
0.14 0.14
0.15 0.15
True self-employed
False 2
False 3
105.0 North 25556.0
0.1975 0.1975
0.14 0.14
0.14 0.14
False salaried
True 2
False 3
708.0 North 18000.0
0.165 0.16
0.15 0.15
0.14 0.14
0.14 0.14
0.14 0.14
False salaried
Fal

False 3
755.0 North 23613.33333333333
0.155 0.15
0.15 0.15
0.16 0.16
False government
False 2
True 3
696.0 North 46326.0
0.165 0.16
0.14 0.14
True self-employed
False 2
False 3
735.0 North 33333.0
0.155 0.15
0.15 0.15
True self-employed
False 2
False 3
727.0 North 45529.0
0.165 0.16
0.14 0.14
0.15 0.15
0.16 0.16
False salaried
False 2
False 3
784.0 North 56169.33333333334
0.15 0.15
0.14 0.14
0.165 0.16
0.14 0.14
0.16 0.16
True self-employed
False 2
False 3
768.0 North 40000.0
0.155 0.15
0.165 0.16
False salaried
False 2
False 3
778.0 North 30000.0
0.16 0.16
0.14 0.14
False salaried
False 2
False 3
776.0 North 30000.0
0.16 0.16
0.14500000000000002 0.14
0.16 0.16
0.14 0.14
0.14 0.14
True self-employed
False 2
False 3
761.0 North 45270.5
0.165 0.16
0.16 0.16
0.15 0.15
False salaried
False 2
False 3
771.0 North 810160.0
0.15 0.15
False salaried
False 2
False 3
710.0 North 61464.0
0.17 0.17
True self-employed
False 2
True 3
695.0 North 67815.0
0.165 0.16
True self-employed
False 2
False 3
7

787.0 North 100000.0
0.14 0.14
0.14 0.14
0.14500000000000002 0.14
0.15 0.15
True self-employed
False 2
False 3
761.0 North 45270.5
0.17 0.17
False salaried
False 2
False 3
804.0 North 51271.0
0.15 0.15
0.15 0.15
0.165 0.16
0.15 0.15
0.165 0.16
False salaried
False 2
False 3
743.0 North 42494.0
0.15 0.15
0.14 0.14
0.16 0.16
0.17 0.17
0.15 0.15
0.155 0.15
True self-employed
False 2
False 3
763.0 North 31666.0
0.17 0.17
0.15 0.15
0.14 0.14
True self-employed
False 2
False 3
775.0 North 30000.0
0.14500000000000002 0.14
0.17 0.17
0.16 0.16
0.15 0.15
0.14 0.14
False salaried
True 2
False 3
128.0 North 20200.0
0.17 0.17
0.15 0.15
0.155 0.15
0.14 0.14
False salaried
False 2
False 3
741.0 North 30000.0
0.17 0.17
0.17 0.17
False salaried
False 2
False 3
751.0 North 30880.0
0.15 0.15
False salaried
False 2
False 3
763.0 North 36000.0
0.14 0.14
0.15 0.15
0.16 0.16
True self-employed
False 2
False 3
751.0 North 48147.0
0.155 0.15
0.14 0.14
0.16 0.16
0.15 0.15
True self-employed
False 2
False 3
738.

0.16 0.16
False salaried
True 2
False 3
744.0 North 25000.0
0.165 0.16
0.17 0.17
True self-employed
False 2
True 3
683.0 North 33333.0
0.17 0.17
0.16 0.16
0.16 0.16
0.15 0.15
0.165 0.16
False salaried
False 2
False 3
740.0 North 40000.0
0.16 0.16
0.14 0.14
False salaried
True 2
False 3
738.0 North 14193.0
0.165 0.16
True self-employed
False 2
False 3
757.0 North 40000.0
0.17 0.17
False salaried
False 2
False 3
737.0 North 52207.66666666666
0.15 0.15
False salaried
False 2
False 3
752.0 North 40000.0
0.16 0.16
True self-employed
False 2
False 3
749.0 North 33333.0
0.14500000000000002 0.14
0.165 0.16
0.15 0.15
False salaried
True 2
False 3
765.0 North 17504.0
0.155 0.15
0.15 0.15
0.17 0.17
0.14 0.14
0.16 0.16
True self-employed
False 2
False 3
758.0 North 33333.0
0.165 0.16
0.165 0.16
0.15 0.15
False salaried
False 2
False 3
793.0 North 78916.66666666669
0.14 0.14
False salaried
False 2
False 3
731.0 North 71753.0
0.17 0.17
0.17 0.17
0.15 0.15
0.15 0.15
False salaried
True 2
False 3
746.

0.165 0.16
True self-employed
False 2
False 3
777.0 North 24995.0
0.155 0.15
0.14 0.14
0.15 0.15
False salaried
True 2
False 3
739.0 North 16425.0
0.155 0.15
0.155 0.15
False salaried
False 2
False 3
789.0 North 35341.5
0.14 0.14
0.15 0.15
0.14 0.14
True self-employed
False 2
False 3
736.0 North 157504.0
0.17 0.17
0.17 0.17
0.16 0.16
False salaried
False 2
False 3
708.0 North 56956.0
0.1975 0.1975
0.165 0.16
0.155 0.15
True self-employed
False 2
False 3
754.0 North 25000.0
0.165 0.16
True self-employed
False 2
False 3
738.0 North 33333.0
0.165 0.16
0.14 0.14
False salaried
False 2
False 3
710.0 North 51558.0
0.14 0.14
0.17 0.17
False salaried
False 2
False 3
753.0 North 32000.0
0.15 0.15
False salaried
False 2
False 3
762.0 North 30000.0
0.16 0.16
False salaried
False 2
False 3
749.0 North 32835.853
0.14 0.14
0.16 0.16
0.14500000000000002 0.14
0.14 0.14
0.165 0.16
True self-employed
False 2
False 3
733.0 North 18908.0
0.165 0.16
0.15 0.15
0.14 0.14
True self-employed
False 2
True 3
680

...

0.12 2
82663.0
0
0.15 1
70121.0
0
0.15 2
35700.0
0
0.165 2
9771.0
1
0.005
0.14500000000000002 1
36400.0
0
0.15 2
31900.0
0
0.13 2
35000.0
0
0.15 2
19561.0
1
0.005
0.135 2
55000.0
0.005
0.155 3
37500.0
0.005
0.165 1
17105.0
1
0.005
0.125 2
74928.66666666669
0
0.165 2
32752.0
0
0.155 2
27463.5
1
0.005
0.14500000000000002 2
54592.0
0
0.13 2
50000.0
0.005
0.165 3
90000.0
0
0.14 2
40000.0
0.005
0.17500000000000002 3
107000.0
0
0.15 2
25000.0
0.005
0.165 3
23000.0
1
0.005
0.155 2
18332.0
1
0.005
0.185 1
46616.0
0
0.15 2
25333.0
0.005
0.165 3
41666.0
0.005
0.155 3
37601.0
0.005
0.165 1
48164.0
0
0.155 2
70000.0
0
0.14 2
250000.0
0
0.13 2
375000.0
0.005
0.23500000000000001 1
39600.0
0
0.16 2
37500.0
0.005
0.165 3
18796.0
1
0.005
0.185 1
50000.0
0.005
0.165 3
39961.0
0
0.15 2
33333.0
0.005
0.17 3
106270.0
0
0.12 2
33333.0
0.005
0.17500000000000002 3
23190.0
0.005
0.17 3
58740.73066666668
0.005
0.165 1
32842.0
0
0.15 2
140000.0
0
0.14 2
80000.0
0
0.14 2
33000.0
0.005
0.155 3
35000.0
0.005
0.165 

0.165 3
37500.0
0.005
0.17 1
25379.2
1
0.005
0.17 1
38583.0
0
0.15 2
33333.0
0.005
0.165 3
43088.0
0
0.13 2
18942.0
1
-0.005
0.155 2
61967.0
0
0.15 2
33333.0
0.005
0.155 3
25000.0
0.005
0.155 3
109891.0
0
0.12 2
400000.0
0
0.12 2
44939.63600000001
0.005
0.155 1
25000.0
0.005
0.14500000000000002 3
76900.0
0
0.14 2
50158.0
0.005
0.155 3
30000.0
0
0.12 2
29166.0
0.005
0.165 3
83049.0
0.005
0.17500000000000002 3
36000.0
0.005
0.14500000000000002 3
47769.0
0
0.12 2
50000.0
0.005
0.165 1
56325.0
0
0.14 2
22374.0
1
0.005
0.14500000000000002 2
30000.0
0.005
0.165 3
100000.0
0
0.14 2
60000.0
0.005
0.165 3
50174.688571428575
0
0.14 2
41817.4
0
0.18 1
53349.33333333334
0
0.15 2
44046.83333333334
0
0.12 2
35338.0
0
0.16 2
100000.0
0
0.15 2
25273.0
1
0.005
0.14500000000000002 2
17122.0
1
0.005
0.155 2
35000.0
0
0.18 2
236877.20400000003
0.005
0.17500000000000002 3
90000.0
0
0.12 2
84816.0
0
0.15 2
20000.0
1
0.005
0.125 2
28844.0
1
0.005
0.17 2
33333.0
0.005
0.17500000000000002 3
43110.11111111112
0

40000.0
0
0.16 2
27000.0
1
0.005
0.155 2
25000.0
1
0.005
0.155 2
14752.0
1
0.005
0.155 2
42513.0
0
0.15 2
33881.0
0
0.15 1
98557.0
0.005
0.17500000000000002 3
59593.8
0
0.12 2
44250.0
0
0.12 2
33333.0
0.005
0.185 1
45000.0
0
0.16 2
30866.66666666667
-0.01
0.15 1
25322.0
1
0.005
0.165 1
33333.0
0.005
0.17500000000000002 3
25000.0
0.005
0.155 3
54914.0
0
0.13 2
18000.0
1
0.005
0.125 2
36083.0
0
0.15 1
33333.0
0.005
0.17 3
31500.0
0.005
0.165 3
15626.0
1
0.005
0.125 2
19866.666666666668
1
0.005
0.14500000000000002 2
33333.0
0.005
0.17 3
13000.0
1
0.005
0.155 2
24000.0
1
0.005
0.155 1
50155.0
0
0.15 2
33333.0
0.005
0.165 1
78000.0
0
0.15 2
46706.0
0
0.14 2
22350.0
1
0.005
0.155 2
17602.0
1
0.005
0.155 1
25689.83333333333
1
0.005
0.17500000000000002 2
18535.666666666668
1
0.005
0.155 1
40000.0
0
0.15 2
50000.0
0
0.15 2
55000.0
0
0.14 2
46420.0
0
0.15 1
25766.0
1
0.005
0.165 2
33814.0
-0.01
0.15 2
70000.0
0
0.15 2
19296.0
1
0.005
0.165 1
43651.0
0
0.13 2
85356.0
0
0.14 2
42668.0
0
0.165 2
59

0.165 2
12201.0
1
0.005
0.125 2
80000.0
-0.01
0.15 1
33333.0
0.005
0.165 3
68666.0
0
0.23 1
46000.0
0
0.15 2
16700.0
1
0.005
0.185 1
17500.0
1
0.005
0.155 2
46344.0
0.005
0.13 3
53861.0
0
0.23 1
73788.0
0
0.15 2
40000.0
0
0.12 2
56656.0
0.005
0.23500000000000001 1
35000.0
0
0.15 2
16812.0
1
0.005
0.155 1
28357.0
0.005
0.165 3
33333.0
0.005
0.155 3
25000.0
0.005
0.185 1
45000.0
0.005
0.165 3
50421.0
0
0.14 2
118000.0
0
0.12 2
30882.0
0
0.15 1
42512.0
0
0.18 1
50000.0
-0.005
0.155 3
20393.0
1
0.005
0.14500000000000002 2
25000.0
0.005
0.17 3
81142.0
0.005
0.165 3
28853.0
1
0.005
0.135 2
41666.0
0.005
0.17 3
25142.0
1
0.005
0.125 2
50000.0
0
0.14 2
21531.0
1
0.005
0.165 2
50361.0
0
0.13 2
41000.0
0
0.12 2
59003.2
0
0.13 2
33333.0
0.005
0.165 1
113098.0
0
0.12 2
7940.0
1
0.005
0.125 2
101577.322
0.005
0.155 3
44969.0
0.005
0.155 3
38500.0
0
0.15 2
31675.0
0.005
0.165 3
100000.0
0.005
0.155 3
17047.0
1
0.005
0.125 2
50000.0
0
0.16 1
119410.0
0.005
0.165 3
75000.0
0.005
0.165 1
33333.0
0.005


32500.0
0
0.16 2
29330.25
1
0.005
0.135 2
72222.22222222223
0
0.15 2
40000.0
0
0.12 2
20000.0
1
0.005
0.17500000000000002 2
40000.0
0.005
0.155 3
28881.0
1
0.005
0.185 1
75163.0
0
0.12 2
45055.0
0
0.14 2
32030.0
0
0.17 2
30596.0
0
0.13 2
25233.33333333333
0.005
0.165 3
84610.0
0
0.14 2
33333.0
0.005
0.17500000000000002 3
41667.0
0.005
0.155 3
38920.0
0
0.16 1
30799.8
0
0.14 2
25000.0
0.005
0.165 1
80000.0
0.005
0.14500000000000002 3
12143.0
1
0.005
0.155 1
50000.0
0.005
0.14500000000000002 3
75000.0
0.005
0.155 3
30000.0
0
0.14 2
23452.0
1
0.005
0.155 2
25000.0
0.005
0.14500000000000002 1
64271.0
0
0.16 2
24500.0
1
0.005
0.17500000000000002 2
25000.0
0.005
0.165 3
134513.44133333335
0.005
0.185 1
30000.0
0
0.16 2
5193.0
0.005
0.17500000000000002 3
70000.0
0
0.14 2
68869.0
0
0.15 2
58797.0
0.005
0.17500000000000002 3
39579.0
0
0.15 1
25000.0
0.005
0.23500000000000001 1
40000.0
0
0.16 2
38000.0
0
0.17 2
65332.0
0
0.14 2
27894.0
1
0.005
0.125 2
50000.0
0.005
0.17500000000000002 3
22000.0


In [23]:
data_df=pd.concat([data_df_jan,data_df_feb_march],axis=0)
data_df.shape

(5811, 97)

In [24]:
wbr_data=pd.read_csv("C:\\Users\\user\\Downloads\\C2C Conversion Deviation Report With Casewise - Raw_Data.csv",
            usecols=['LEAD_ID','ds_roi','ROI_Deviation','Final_ROI','last_risk_bucket','PF_Deviation','CHM_Deviation','Overall_Deviation'])
wbr_data.rename({'ds_roi':'ds_roi_wbr','ROI_Deviation':'ROI_Deviation_wbr','PF_Deviation':"PF_Deviation_wbr",'CHM_Deviation':'CHM_Deviation_wbr','Overall_Deviation':'Overall_Deviation_wbr'},axis=1,inplace=True)
wbr_data['ds_roi_wbr']=pd.to_numeric(wbr_data['ds_roi_wbr'])

In [25]:
data_df=data_df.merge(wbr_data,on='LEAD_ID',how='left')

# conversion deviation % (only -ve deviation)

In [26]:
data_df['ROI_Deviation_wbr'].value_counts(dropna=False)

0.0    4816
NaN     511
1.0     488
Name: ROI_Deviation_wbr, dtype: int64

In [27]:
print(data_df['ROI_Deviation_wbr'].sum()/len(data_df))

0.08392089423903697


# Removing risk bucket diff cases

In [28]:
data_df['rb_equal_flag']=np.where(data_df['last_risk_bucket']==data_df['FINAL_PC_RB_V4_AJ'],1,0)
data_df['rb_equal_flag'].value_counts()

1    5202
0     613
Name: rb_equal_flag, dtype: int64

In [29]:
print(data_df[(data_df['rb_equal_flag']==0) & ~(data_df['last_risk_bucket'].isnull())][['LEAD_ID','last_risk_bucket','FINAL_PC_RB_V4_AJ']].shape)
data_df=data_df[~((data_df['rb_equal_flag']==0) & ~(data_df['last_risk_bucket'].isnull()))]

(101, 3)


# cases with ds roi mismatch

In [30]:
data_df['DS_BI_ROI2']=data_df['DS_BI_ROI']*100
data_df['ds_bi_roi_vs_ds_roi']=np.where(np.round(data_df['DS_BI_ROI2'],1)==np.round(data_df['ds_roi_wbr'],1),1,0)
data_df['ds_bi_roi_vs_ds_roi'].value_counts()

1    4852
0     862
Name: ds_bi_roi_vs_ds_roi, dtype: int64

In [31]:
z2=data_df[data_df['ds_bi_roi_vs_ds_roi']==0]

In [32]:
data_df['ds_bi_roi_vs_ds_roi_diff']=np.round(data_df['DS_BI_ROI2'],1)-np.round(data_df['ds_roi_wbr'],1)

In [33]:
data_df[(data_df['ds_bi_roi_vs_ds_roi']==0) & ((data_df['ds_bi_roi_vs_ds_roi_diff']>=0.5) |(data_df['ds_bi_roi_vs_ds_roi_diff']<=-0.5)  )][['LEAD_ID','AGREEMENT_DATE','ds_roi_wbr','ds_bi_roi_vs_ds_roi_diff','DS_BI_ROI2','FINAL_PC_RB_V4_AJ','region_pincode','APP_AVG_CIBIL_SCORE','PRIMARY_EMP_TYPE','APP_COMB_INCOME','CIBIL_MASTER_DATA2','INCOME_MASTER_DATA2','APP_AQB_MASTER','AQB_BY_EMI','APP_AQB']].shape

(290, 15)

In [34]:
290/4302

0.06741050674105067

In [35]:
cases=data_df[(data_df['ds_bi_roi_vs_ds_roi']==0) & ((data_df['ds_bi_roi_vs_ds_roi_diff']>=0.5) |(data_df['ds_bi_roi_vs_ds_roi_diff']<=-0.5)  )]
cases.shape

(290, 108)

In [36]:
cases[['LEAD_ID','AGREEMENT_DATE','ds_roi_wbr','ds_bi_roi_vs_ds_roi_diff','DS_BI_ROI2','FINAL_PC_RB_V4_AJ','region_pincode','APP_AVG_CIBIL_SCORE','PRIMARY_EMP_TYPE','APP_COMB_INCOME','CIBIL_MASTER_DATA2','INCOME_MASTER_DATA2','APP_AQB_MASTER','AQB_BY_EMI','APP_AQB','APK_DECISION_CATEGORY_FIRST']].to_csv("tmp2.csv",index=False)

In [37]:
cases['AGREEMENT_MONTH'].value_counts()

2022-04    100
2022-05     90
2022-03     50
2022-02     26
2022-01     24
Name: AGREEMENT_MONTH, dtype: int64

In [38]:
data_df=data_df[~((data_df['ds_bi_roi_vs_ds_roi']==0) & ((data_df['ds_bi_roi_vs_ds_roi_diff']>=0.5) |(data_df['ds_bi_roi_vs_ds_roi_diff']<=-0.5)  ))]

In [39]:
data_df.shape

(5424, 108)

In [40]:
data_df.groupby(['AGREEMENT_MONTH']).size()

AGREEMENT_MONTH
2022-01     955
2022-02    1063
2022-03    1220
2022-04    1180
2022-05    1006
dtype: int64

# deriving DS+BI ROI dev Metrics

In [41]:
replace_percent={9: {'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
 10:{'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
11:{'A':14,'B':15,'C':16,'D':17,'E':21},
12:{'A':14,'B':15,'C':16,'D':17,'E':21},
1:{'A':14,'B':15,'C':16,'D':16.5,'E':19.5},
2:{'A':14,'B':15,'C':16,'D':16.5,'E':19.75},
4:{'A':14,'B':15,'C':16,'D':17, 'E':19.75}}

def bi_assign_baseroi(row):
    if row['AGREEMENT_MONTH']=='2021-09':
        return replace_percent[9][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-10':
        return replace_percent[10][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-11':
        return replace_percent[11][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2021-12':
        return replace_percent[12][row['FINAL_PC_RB_V4_AJ']]
    elif ((row['AGREEMENT_MONTH']=='2021-12') & (row['AGREEMENT_day']<=3)):
        return replace_percent[12][row['FINAL_PC_RB_V4_AJ']]
    elif ((row['AGREEMENT_MONTH']=='2022-01') & (row['AGREEMENT_day']>3)):
        return replace_percent[1][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-02' :
        return replace_percent[2][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-03' :
        return replace_percent[2][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-04' :
        return replace_percent[4][row['FINAL_PC_RB_V4_AJ']]
    elif row['AGREEMENT_MONTH']=='2022-05' :
        return replace_percent[4][row['FINAL_PC_RB_V4_AJ']]
    else:
        return 0
data_df['baseroi']=data_df.apply(bi_assign_baseroi, axis=1)

data_df['delta_roi_ds_bi']=(data_df['DS_BI_ROI2'])-data_df['baseroi']
# data_df['delta_roi_ds_bi']=data_df['delta_roi_ds_bi']

def bin_assign(a):
    if a<-0.25 :
        return 0
    elif a>0.25:
        return 2
    else:
        return 1
data_df['delta_roi_bin_binary_ds_bi']=data_df.apply(lambda x: bin_assign(x['delta_roi_ds_bi']), axis=1)
data_df['delta_roi_bin_binary_ds_bi'].value_counts(dropna=False,sort=False)

0     550
1    4640
2     234
Name: delta_roi_bin_binary_ds_bi, dtype: int64

In [42]:
data_df[['delta_roi_ds_bi','DISBURSED_ROI','DS_BI_ROI2']].head()

,delta_roi_ds_bi,DISBURSED_ROI,DS_BI_ROI2
0,0.000000e+00,0.140,14.0
1,-3.552714e-15,0.145,14.5
2,0.000000e+00,0.140,14.0
3,0.000000e+00,0.170,17.0
4,0.000000e+00,0.150,15.0


# add pf deviaiton

In [43]:
data_df["pf_percent_disbursed"]=pd.to_numeric(data_df['PF'])/pd.to_numeric(data_df['CLA'])
data_df[["pf_percent_disbursed",'PF','CLA']].head()

,pf_percent_disbursed,PF,CLA
0,0.0236,9588.36,406286.48
1,0.0236,12746.16,540091.57
2,0.0236,3652.01,154746.10
3,0.0236,11147.36,472345.83
4,0.0236,11522.02,488221.15


In [44]:
ds_df=pd.read_feather("../ds_roi_data_25_05_2022.feather")
data_df=data_df.merge(ds_df[['LEAD_ID','PF_PERCENT_DS','LI_ds_rb', 'gps_charges_ds_rb','GPS_COST_disbursed', 'LOAN_INSURANCE_disbursed','MOTOR_INSURANCE_disbursed']],on='LEAD_ID',how='left')

In [45]:
data_df['pf_per_tmp']=data_df['pf_percent_disbursed'].round(4)
data_df['pf_per_tmp'].value_counts().head(8)

0.0236    4039
0.0118     970
0.0177     194
0.0354      27
0.0201      18
0.0212      16
0.0153      13
0.0000      12
Name: pf_per_tmp, dtype: int64

In [46]:
data_df['PF_PERCENT_DS'].value_counts()

0.02     5112
0.03       49
0           1
0.015       1
Name: PF_PERCENT_DS, dtype: int64

In [47]:
data_df['PF_PERCENT_DS_tmp']=pd.to_numeric(data_df['PF_PERCENT_DS'])*1.18
data_df['PF_PERCENT_DS_tmp']=data_df['PF_PERCENT_DS_tmp'].round(4)
data_df['PF_PERCENT_DS_tmp'].value_counts(dropna=False)

0.0236    5112
NaN        261
0.0354      49
0.0177       1
0.0000       1
Name: PF_PERCENT_DS_tmp, dtype: int64

###  GPS charges

In [48]:
data_df['GPS_COST_disbursed'].value_counts(normalize=True)

4999.00    0.823746
2999.00    0.084808
5000.00    0.038901
3000.00    0.012353
3999.00    0.011062
2500.00    0.005900
1999.00    0.005162
2499.00    0.004794
0.00       0.004056
4000.00    0.003503
3499.00    0.001659
2000.00    0.001475
3500.00    0.001475
1000.00    0.000369
1500.00    0.000184
500.00     0.000184
2200.00    0.000184
3600.00    0.000184
Name: GPS_COST_disbursed, dtype: float64

In [49]:
data_df['gps_diff_delta']=data_df['GPS_COST_disbursed']-4999
data_df['gps_diff_delta'].value_counts(sort=False)

0.00        4468
1.00         211
-1000.00      60
-999.00       19
-2000.00     460
-1999.00      67
-3000.00      28
-2999.00       8
-3999.00       2
-4999.00      22
-2799.00       1
-1500.00       9
-1499.00       8
-2500.00      26
-2499.00      32
-3499.00       1
-4499.00       1
-1399.00       1
Name: gps_diff_delta, dtype: int64

In [50]:
data_df['gps_charges_ds_rb'].value_counts()

4999    5162
Name: gps_charges_ds_rb, dtype: int64

In [51]:
data_df['GPS_COST_disbursed'].isnull().sum()

0

In [52]:
def f(row):
    ret_val=0
    if (row['PF_PERCENT_DS_tmp']==0.0236 ) & (row['pf_per_tmp']<0.0236):
        ret_val=1
    elif (row['PF_PERCENT_DS_tmp']==0.0354 ) & (row['pf_per_tmp']<0.0354):
        ret_val=1
    if (row['gps_diff_delta']<-800):
        if ret_val==0:
            ret_val=1
        
    return ret_val
data_df['PF_gps_DISCOUNT_FLAG'] = data_df.apply(f, axis=1)
data_df['PF_gps_DISCOUNT_FLAG'].value_counts()

0    3812
1    1612
Name: PF_gps_DISCOUNT_FLAG, dtype: int64

In [53]:
print(data_df['Overall_Deviation_wbr'].sum(),data_df['PF_Deviation_wbr'].sum(),data_df['CHM_Deviation_wbr'].sum())

1652.0 1163.0 653.0


In [54]:
print(data_df['Overall_Deviation_wbr'].sum(),data_df['PF_gps_DISCOUNT_FLAG'].sum())

1652.0 1612


In [55]:
def f1(row):
    ret_val=0
    if (row['PF_PERCENT_DS_tmp']==0.0236 ) & (row['pf_per_tmp']<0.0236):
        ret_val=1
    elif (row['PF_PERCENT_DS_tmp']==0.0354 ) & (row['pf_per_tmp']<0.0354):
        ret_val=1
    return ret_val

def f2(row):
    ret_val=0
    
    if (row['gps_diff_delta']<-800):
            ret_val=1
    
    return ret_val


data_df['PF_DISCOUNT_FLAG'] = data_df.apply(f1, axis=1)
data_df['GPS_DISCOUNT_FLAG'] = data_df.apply(f2, axis=1)
# data_df['PF_DISCOUNT_FLAG'].value_counts()

In [56]:
data_df[['LEAD_ID','PF_Deviation_wbr','PF_gps_DISCOUNT_FLAG','PF_PERCENT_DS_tmp','pf_per_tmp','gps_diff_delta','CHM_Deviation_wbr','Overall_Deviation_wbr','AGREEMENT_MONTH','PF_DISCOUNT_FLAG','GPS_DISCOUNT_FLAG']].to_csv("tmp2.csv",index=False)

### correcting class labels

In [ ]:
#simulated data
data_df['delta_roi_bin_binary_ds_bi_orig']=data_df['delta_roi_bin_binary_ds_bi']
def label_corr(row):
    #no discount into discount grp
    if (row['delta_roi_bin_binary_ds_bi']==2 ) & (row['PF_gps_DISCOUNT_FLAG']==1):
        return 1
    #
    elif (row['delta_roi_bin_binary_ds_bi']==1 ) & (row['PF_gps_DISCOUNT_FLAG']==1):
        return 0
    else:
        return row['delta_roi_bin_binary_ds_bi']
    
data_df['delta_roi_bin_binary_ds_bi'] = data_df.apply(label_corr, axis=1)
data_df['delta_roi_bin_binary_ds_bi'].value_counts(sort=False)

In [57]:
data_df['delta_roi_bin_binary_orig']=data_df['delta_roi_bin_binary']

def label_corr(row):
    #no discount into discount grp
    if (row['delta_roi_bin_binary']==2 ) & (row['PF_gps_DISCOUNT_FLAG']==1):
        return 1
    #
    elif (row['delta_roi_bin_binary']==1 ) & (row['PF_gps_DISCOUNT_FLAG']==1):
        return 0
    else:
        return row['delta_roi_bin_binary']
    
data_df['delta_roi_bin_binary'] = data_df.apply(label_corr, axis=1)
data_df['delta_roi_bin_binary'].value_counts(sort=False)

0    1845
1    1718
2    1861
Name: delta_roi_bin_binary, dtype: int64

In [58]:
data_df['delta_roi_bin_binary'].value_counts(normalize=True)

2    0.343105
0    0.340155
1    0.316740
Name: delta_roi_bin_binary, dtype: float64

In [59]:
data_df['delta_roi_bin_binary_orig'].value_counts(normalize=True)

2    0.454277
1    0.285951
0    0.259771
Name: delta_roi_bin_binary_orig, dtype: float64

# CONVERSION METRICS

In [60]:
# % roi deviation 
# % upfront deviation
# % overall deviation= (roi+upfront)

#roi deviation
print(data_df['ROI_Deviation_wbr'].sum(),len(data_df),data_df['ROI_Deviation_wbr'].sum()/len(data_df))

446.0 5424 0.08222713864306784


In [61]:
# base_ds
# base+bi-->>

# DS METRICS

In [62]:
# selling with discount
print(len(data_df[data_df['delta_roi_bin_binary_ds_bi']==0]),\
      len(data_df),data_df[data_df['delta_roi_bin_binary_ds_bi']==0].shape[0]/len(data_df))

550 5424 0.10140117994100295


In [63]:
# selling with premium+ discount

print(len(data_df[(data_df['delta_roi_bin_binary_ds_bi']==0)| (data_df['delta_roi_bin_binary_ds_bi']==2)  ]),\
      len(data_df),data_df[(data_df['delta_roi_bin_binary_ds_bi']==0)| (data_df['delta_roi_bin_binary_ds_bi']==2)  ].shape[0]/len(data_df))

784 5424 0.14454277286135694


# data prep for model

In [64]:
#     if a<-0.25:
#         return 0
#     elif a>0.25:
#         return 2
#     else:
#         return 1

In [65]:
data_df['delta_roi_model1']=data_df['delta_roi_bin_binary']
data_df['delta_roi_model1']=data_df['delta_roi_model1'].replace({0:11,1:22,2:22})
data_df['delta_roi_model1'].value_counts()

22    3579
11    1845
Name: delta_roi_model1, dtype: int64

In [66]:
data_df['delta_roi_model1']=data_df['delta_roi_model1'].replace({11:1,22:0,22:0})
data_df['delta_roi_model1'].value_counts()

0    3579
1    1845
Name: delta_roi_model1, dtype: int64

In [67]:
data_df.groupby(['AGREEMENT_MONTH','delta_roi_bin_binary']).agg({'delta_roi_model1':'sum','LEAD_ID':'count'})

delta_roi_model1  LEAD_ID
AGREEMENT_MONTH delta_roi_bin_binary                           
2022-01         0                                  176      176
                1                                    0      417
                2                                    0      362
2022-02         0                                  271      271
                1                                    0      453
                2                                    0      339
2022-03         0                                  491      491
                1                                    0      293
                2                                    0      436
2022-04         0                                  480      480
                1                                    0      314
                2                                    0      386
2022-05         0                                  427      427
                1                                    0      241
                2                                    0      338

In [68]:
data_df['tmp']=data_df['AGREEMENT_DATE'].astype(str).str[:4]
data_df['car_age']=pd.to_numeric(data_df['tmp'])-pd.to_numeric(data_df['CAR_YEAR'])

In [69]:
data_df['cobo_flag'] = np.where(data_df['cobo_age'].isnull() ,0,1)
data_df['cobo_flag'].value_counts()

0    5094
1     330
Name: cobo_flag, dtype: int64

In [70]:
data_df['aqb_emi_gt_05_flag'] = np.where(data_df['AQB_BY_EMI']>0.5 ,1,0)
data_df['aqb_emi_gt_05_flag'].value_counts()

1    4250
0    1174
Name: aqb_emi_gt_05_flag, dtype: int64

In [71]:
# exhaustive feature list
all_cols=['LEAD_ID', 'PF', 'DLA', 'CLA', 'TLA', 'LOAN_PROCESS',
       'CAR_FINAL_VALUE', 'LEAD_CREATE_DATE', 'AGREEMENT_FLAG',
       'OK_FOR_DISB_FLAG', 'DISBURSAL_FLAG', 
       'AGREEMENT_DATE', 'DELIVERY_DATE', 'DISBURSED_TENURE',
       'FINANCEOPTED',
        'PRIMARY_EMP_TYPE', 'PRIMARY_NIP_FLAG',
       'PRIMARY_ACCOMODATION_STATUS', 'PRIMARY_GENDER', 'PRIMARY_DOB',
       'PRIMARY_OCCUPATION', 'PRIMARY_RESIDENT_STABILITY',
       'PRIMARY_TOTAL_WORK_EX', 'PRIMARY_EDUCATION_BACKGROUND',
       'PRIMARY_MARITAL_STATUS', 'PRIMARY_EXEC_DEF_MONTHLY_INCOME',
       'PRIMARY_EXEC_DEF_AVG_CREDITS', 'PRIMARY_NET_INCOME',
       'PRIMARY_EXEC_DEF_EMI', 'PRIMARY_PERFIOS_ESTIMATED_EMI',
       'PRIMARY_NET_EMI', 'PRIMARY_NET_AQB', 'PRIMARY_CIBIL_SCORE',
       'COBO_EMP_TYPE', 'COBO_EMPLOYMENT_DETAILS', 'COBO_NIP_FLAG',
       'COBO_ACCOMODATION_STATUS', 'COBO_GENDER',
       'COBO_EXEC_DEF_MONTHLY_INCOME', 'COBO_EXEC_DEF_EMI',
       'COBO_EXEC_DEF_AQB', 'COBO_CIBIL_SCORE', 'APP_AVG_CIBIL_SCORE',
       'APP_CIBIL_EMI', 'APP_NET_EMI', 'APP_NIP_FLAG', 'APP_HOUSE_OWNERSHIP',
       'CITY_CAR_PURCHASE', 'STATE', 'CITY', 'PINCODE ', 'APP_COMB_INCOME',
       'FUEL_TYPE', 'MAKE', 'MODEL', 'VARIANT_INSPECTION', 'CAR_YEAR',
       'CAR_OWNER_NUMBER', 'Total_TLV', 'Disbursal_TLV', 'APP_FOIR',
       'CFSPL_EMI', 'PRIMARY_CURRENT_FOIR', 'PRIMARY_FOIR_INC_PROP_EMI',
       'cobo_age', 'primary_age', 'APP_AGE', 'AGREEMENT_MONTH','car_age','cobo_flag','aqb_emi_gt_05_flag','CIBIL_MASTER_DATA2',
         'INCOME_MASTER_DATA2','FINAL_PC_RB_V4_AJ','AQB_DERIVED']

cols_to_drop=['LEAD_ID','LOAN_PROCESS','LEAD_CREATE_DATE' ,'DISBURSAL_FLAG','AGREEMENT_DATE','AGREEMENT_FLAG','DELIVERY_DATE',
              'PRIMARY_DOB','PINCODE ','AGREEMENT_MONTH','CAR_YEAR','TENURE','CHANNEL_MAIN', 'SUB_CHANNEL',
               'FINANCEOPTED','OK_FOR_DISB_FLAG',
              'PRIMARY_PERFIOS_ESTIMATED_EMI','APP_CIBIL_EMI','GEO_LIMIT','DS_TENURE_V3']

print(len(all_cols))
all_cols=[e for e in all_cols if e not in cols_to_drop]
print(len(all_cols))

75
60


In [72]:
data_df['delta_roi_model1'].value_counts()

0    3579
1    1845
Name: delta_roi_model1, dtype: int64

# correcting data types and cleaning categorical features

In [73]:
cleanup_cat_cols = {"FINAL_PC_RB_V4_AJ":{'"A"':1,'"B"':2,'"C"':3,'"D"':4,'"E"':5}}
data_df=data_df.replace(cleanup_cat_cols)

In [74]:
for cols in ['PF', 'FINANCEOPTED', 'PRIMARY_RESIDENT_STABILITY', 'PRIMARY_TOTAL_WORK_EX',
         'PRIMARY_PERFIOS_ESTIMATED_EMI',
      'APP_AVG_CIBIL_SCORE', 'APP_CIBIL_EMI',
      'CAR_YEAR', 'CAR_OWNER_NUMBER']:
    print(cols)
    data_df[cols]=data_df[cols].astype('float64')

PF
FINANCEOPTED
PRIMARY_RESIDENT_STABILITY
PRIMARY_TOTAL_WORK_EX
PRIMARY_PERFIOS_ESTIMATED_EMI
APP_AVG_CIBIL_SCORE
APP_CIBIL_EMI
CAR_YEAR
CAR_OWNER_NUMBER


In [75]:
len(all_cols)

60

In [76]:
data_df.groupby(['AGREEMENT_MONTH']).size()

AGREEMENT_MONTH
2022-01     955
2022-02    1063
2022-03    1220
2022-04    1180
2022-05    1006
dtype: int64

# treat column

In [77]:
category_columns = []
object_columns = []
numeric_columns=[]
 
for cols in all_cols:
    if data_df[cols].dtype.name == 'category':
        category_columns.append(cols)
    elif data_df[cols].dtype.name == 'object':
        object_columns.append(cols)
    elif data_df[cols].dtype.name in ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']:
         numeric_columns.append(cols) 
    
    else:
        continue

In [78]:
print (len(object_columns))
print (len(category_columns))
print (len(numeric_columns))

22
0
38


In [79]:
numeric_columns

['PF',
 'DLA',
 'CLA',
 'TLA',
 'CAR_FINAL_VALUE',
 'DISBURSED_TENURE',
 'PRIMARY_RESIDENT_STABILITY',
 'PRIMARY_TOTAL_WORK_EX',
 'PRIMARY_EXEC_DEF_MONTHLY_INCOME',
 'PRIMARY_EXEC_DEF_AVG_CREDITS',
 'PRIMARY_NET_INCOME',
 'PRIMARY_EXEC_DEF_EMI',
 'PRIMARY_NET_EMI',
 'PRIMARY_NET_AQB',
 'PRIMARY_CIBIL_SCORE',
 'COBO_EXEC_DEF_MONTHLY_INCOME',
 'COBO_EXEC_DEF_EMI',
 'COBO_EXEC_DEF_AQB',
 'COBO_CIBIL_SCORE',
 'APP_AVG_CIBIL_SCORE',
 'APP_NET_EMI',
 'APP_COMB_INCOME',
 'CAR_OWNER_NUMBER',
 'Total_TLV',
 'Disbursal_TLV',
 'APP_FOIR',
 'CFSPL_EMI',
 'PRIMARY_CURRENT_FOIR',
 'PRIMARY_FOIR_INC_PROP_EMI',
 'cobo_age',
 'primary_age',
 'APP_AGE',
 'car_age',
 'cobo_flag',
 'aqb_emi_gt_05_flag',
 'CIBIL_MASTER_DATA2',
 'INCOME_MASTER_DATA2',
 'AQB_DERIVED']

In [80]:
object_columns

['PRIMARY_EMP_TYPE',
 'PRIMARY_NIP_FLAG',
 'PRIMARY_ACCOMODATION_STATUS',
 'PRIMARY_GENDER',
 'PRIMARY_OCCUPATION',
 'PRIMARY_EDUCATION_BACKGROUND',
 'PRIMARY_MARITAL_STATUS',
 'COBO_EMP_TYPE',
 'COBO_EMPLOYMENT_DETAILS',
 'COBO_NIP_FLAG',
 'COBO_ACCOMODATION_STATUS',
 'COBO_GENDER',
 'APP_NIP_FLAG',
 'APP_HOUSE_OWNERSHIP',
 'CITY_CAR_PURCHASE',
 'STATE',
 'CITY',
 'FUEL_TYPE',
 'MAKE',
 'MODEL',
 'VARIANT_INSPECTION',
 'FINAL_PC_RB_V4_AJ']

In [81]:
cleanup_cat_cols = {"PRIMARY_EMP_TYPE":{'Government':'Salaried'}  ,"COBO_EMP_TYPE":{'Government':'Salaried'}     }
data_df=data_df.replace(cleanup_cat_cols)

In [82]:
# hold_df=data_df[data_df['AGREEMENT_MONTH']<='2021-08']
# data_df=data_df[data_df['AGREEMENT_MONTH']>'2021-08'])

In [83]:
feature_num=numeric_columns

def EDD_Numeric(data,features):
    results=[]
    def get_statistics(feature):
   
        stat={}
        stat['missing_value_rate']=feature.isnull().sum()*100.0/len(feature)
        feature_noNull=feature.dropna()
        if len(feature_noNull)==0:
            stat['#unique_values']=0
            stat['mins']=stat['maxs']=stat['means']=stat['medians']=stat['stds']=stat['p25']=stat['p1']=stat['p5']=stat['p75']=stat['p95']=stat['p99']=None
        else:
            stat['#unique_values']=len(feature_noNull.unique())
            stat['mins'] = min(feature_noNull)
            stat['maxs'] = max(feature_noNull)
            stat['means'] = np.mean(feature_noNull)
#             stat['modes']=mode(feature_noNull)
#             stat['modes_pct']=stat['modes'][1][0]*1.0/len(feature_noNull)
            stat['medians'] = np.median(feature_noNull)
            stat['stds'] = np.std(feature_noNull)
            stat['p25'] = np.percentile(feature_noNull, 25)
            stat['p1'] = np.percentile(feature_noNull, 1 )
            stat['p2'] = np.percentile(feature_noNull, 2 )
            stat['p5'] = np.percentile(feature_noNull, 5)
            stat['p75'] = np.percentile(feature_noNull, 75)
            stat['p95'] = np.percentile(feature_noNull, 95)
            stat['p99'] = np.percentile(feature_noNull, 99)
        return stat
    for i,e in enumerate(features):
#         print (i,e)
        entry=get_statistics(data[e])
        entry['feature_name']=e
        results.append(entry)
    return pd.DataFrame(results)[['feature_name','#unique_values','missing_value_rate','mins','maxs','means','medians','stds','p1','p2','p5','p25','p75','p95','p99']]
EDD_Numeric(data_df,feature_num)

,feature_name,#unique_values,missing_value_rate,mins,maxs,means,medians,stds,p1,p2,p5,p25,p75,p95,p99
0,PF,5344,0.000000,0.000000e+00,2.375466e+04,8.914106e+03,8256.795000,3.766517e+03,2439.778000,2926.066800,3609.758000,6084.287500,11481.995000,15341.140000,1.853218e+04
1,DLA,5394,0.129056,6.090534e+04,1.541429e+06,4.209840e+05,391190.190000,1.731547e+05,147584.456000,165462.222400,199547.412000,292318.520000,520198.300000,725223.196000,9.538949e+05
2,CLA,5392,0.129056,6.237745e+04,1.559835e+06,4.299007e+05,399762.960000,1.761119e+05,150798.762400,169358.562800,204204.804000,298698.690000,531885.530000,739562.664000,9.703727e+05
3,TLA,5343,0.000000,7.004200e+04,1.632062e+06,4.464448e+05,415633.500000,1.807199e+05,160639.710000,180826.000000,215005.400000,311780.500000,550257.000000,766860.150000,9.973174e+05
4,CAR_FINAL_VALUE,3506,0.000000,0.000000e+00,1.893299e+06,5.077062e+05,474349.000000,2.068714e+05,210337.000000,229983.000000,259299.000000,354874.000000,609274.000000,876499.000000,1.225824e+06
5,DISBURSED_TENURE,43,0.000000,0.000000e+00,7.200000e+01,5.529849e+01,60.000000,1.026266e+01,22.000000,24.000000,36.000000,60.000000,60.000000,60.000000,6.000000e+01
6,PRIMARY_RESIDENT_STABILITY,164,0.036873,0.000000e+00,6.508333e+01,8.425796e+00,5.000000,1.016889e+01,0.000000,0.166667,0.666667,2.000000,10.000000,31.000000,4.279000e+01
7,PRIMARY_TOTAL_WORK_EX,155,0.036873,0.000000e+00,5.500000e+01,7.096858e+00,5.000000,6.368245e+00,0.000000,1.000000,1.000000,2.166667,10.000000,20.000000,3.000000e+01
8,PRIMARY_EXEC_DEF_MONTHLY_INCOME,2599,0.000000,0.000000e+00,1.028644e+06,5.376036e+04,40000.000000,5.546061e+04,9138.970000,12508.380000,16702.250000,27819.500000,61178.750000,122311.500000,2.757357e+05
9,PRIMARY_EXEC_DEF_AVG_CREDITS,854,82.577434,0.000000e+00,8.164606e+07,2.808517e+05,109478.000000,2.675755e+06,0.880000,2560.160000,14545.000000,48665.000000,207624.000000,595284.600000,1.496881e+06


In [84]:
# for i in feature_num:
#     upper_limit = data_df[i].quantile(0.99)
#     lower_limit = data_df[i].quantile(0.01)
#     data_df[i] = np.where(
#                          data_df[i]>upper_limit, upper_limit, 
#                          np.where(data_df[i]<lower_limit,lower_limit,data_df[i])
#                             )

In [85]:
oot_df=data_df[data_df['AGREEMENT_MONTH']>'2022-03']
data_df=data_df[data_df['AGREEMENT_MONTH']<='2022-03']

In [86]:
oot_df.shape

(2186, 129)

In [87]:
data_df.shape

(3238, 129)

In [88]:
car_related=['car_age','CAR_FINAL_VALUE','CAR_OWNER_NUMBER','FUEL_TYPE','MAKE','MODEL']
offer=['CFSPL_EMI','Disbursal_TLV','DISBURSED_TENURE']
profile_validated=['APP_AGE','APP_AVG_CIBIL_SCORE','APP_COMB_INCOME','COBO_EXEC_DEF_EMI','primary_age','PRIMARY_CURRENT_FOIR',
    'PRIMARY_EXEC_DEF_AVG_CREDITS','PRIMARY_EXEC_DEF_EMI','PRIMARY_FOIR_INC_PROP_EMI','PRIMARY_NET_EMI',
    'PRIMARY_ACCOMODATION_STATUS','PRIMARY_OCCUPATION','STATE','CITY']
profile_non_validated=['PRIMARY_RESIDENT_STABILITY','PRIMARY_TOTAL_WORK_EX']
keep_columns=profile_validated+profile_non_validated+car_related+offer
data_df[data_df['LEAD_ID']==28134][keep_columns].T

""
APP_AGE
APP_AVG_CIBIL_SCORE
APP_COMB_INCOME
COBO_EXEC_DEF_EMI
primary_age
PRIMARY_CURRENT_FOIR
PRIMARY_EXEC_DEF_AVG_CREDITS
PRIMARY_EXEC_DEF_EMI
PRIMARY_FOIR_INC_PROP_EMI
PRIMARY_NET_EMI


In [89]:
data_df.columns

Index(['LEAD_ID', 'PF', 'DLA', 'CLA', 'TLA', 'TENURE', 'LOAN_PROCESS',
       'CAR_FINAL_VALUE', 'LEAD_CREATE_DATE', 'AGREEMENT_FLAG',
       ...
       'gps_diff_delta', 'PF_gps_DISCOUNT_FLAG', 'PF_DISCOUNT_FLAG',
       'GPS_DISCOUNT_FLAG', 'delta_roi_bin_binary_orig', 'delta_roi_model1',
       'tmp', 'car_age', 'cobo_flag', 'aqb_emi_gt_05_flag'],
      dtype='object', length=129)

In [90]:
data_df.groupby(['AGREEMENT_MONTH','delta_roi_bin_binary']).size()

AGREEMENT_MONTH  delta_roi_bin_binary
2022-01          0                       176
                 1                       417
                 2                       362
2022-02          0                       271
                 1                       453
                 2                       339
2022-03          0                       491
                 1                       293
                 2                       436
dtype: int64

In [91]:
data_df.groupby(['AGREEMENT_MONTH']).size()

AGREEMENT_MONTH
2022-01     955
2022-02    1063
2022-03    1220
dtype: int64

In [92]:
data_0=data_df[data_df['delta_roi_bin_binary']==0]
data_1=data_df[data_df['delta_roi_bin_binary']==1]
data_2=data_df[data_df['delta_roi_bin_binary']==2]

In [93]:
train_0, test_0 = train_test_split(data_0, test_size = 0.3 ) 
train_1, test_1 = train_test_split(data_1, test_size = 0.3 ) 
train_2, test_2 = train_test_split(data_2, test_size = 0.3 ) 

In [94]:
print(train_2.shape,test_2.shape)

(795, 129) (342, 129)


In [95]:
print(pd.concat([train_0,train_1,train_2],axis=0).shape,pd.concat([train_0,train_1,train_2],axis=0).shape)
train=pd.concat([train_0,train_1,train_2],axis=0)
test=pd.concat([test_0,test_1,test_2],axis=0)

(2265, 129) (2265, 129)


In [96]:
print (('train target rate:'),train['delta_roi_model1'].mean())
print (('test target rate:'),test['delta_roi_model1'].mean())

train target rate: 0.28962472406181017
test target rate: 0.289825282631038


In [97]:
pd.concat([test_0,test_1,test_2],axis=1).shape

(973, 387)

In [98]:
# train, test = train_test_split(data_df, test_size = 0.3 ) 
# test, val = train_test_split(test, test_size = 0.33, random_state = 222) 
# round1=0
# while(abs(train['delta_roi_model1'].mean()-test['delta_roi_model1'].mean())>0.001):
#     train, test = train_test_split(data_df, test_size = 0.3) 
# #     print(round1,end=' ')
#     round1+=1
    
print (('train target rate:'),train['delta_roi_model1'].mean())
print (('test target rate:'),test['delta_roi_model1'].mean())

print (('OOT JAN target rate:'),oot_df['delta_roi_model1'].mean())

# print (('val target rate:'),val['delta_roi_bin_binary'].mean())


print ((train['delta_roi_model1'].mean() - test['delta_roi_model1'].mean())/train['delta_roi_model1'].mean())

# print(train.shape,test.shape,val.shape)

train target rate: 0.28962472406181017
test target rate: 0.289825282631038
OOT JAN target rate: 0.4149130832570906
-0.0006924773769833501


In [99]:
oot_df.groupby(['AGREEMENT_MONTH']).size()

AGREEMENT_MONTH
2022-04    1180
2022-05    1006
dtype: int64

In [100]:
print(train['delta_roi_bin_binary'].value_counts(normalize=True)*100)
print(test['delta_roi_bin_binary'].value_counts(normalize=True)*100)
print(oot_df['delta_roi_bin_binary'].value_counts(normalize=True)*100)

1    35.938190
2    35.099338
0    28.962472
Name: delta_roi_bin_binary, dtype: float64
1    35.868448
2    35.149024
0    28.982528
Name: delta_roi_bin_binary, dtype: float64
0    41.491308
2    33.119854
1    25.388838
Name: delta_roi_bin_binary, dtype: float64


### Compute WOE for categorical variables in training data

In [101]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [102]:
nb_name
# ='data_prep_model1_pf_gps_data.ipynb'

NameError: name 'nb_name' is not defined

In [ ]:
train['CITY'].value_counts()

In [ ]:
import os
if ~os.path.isdir('./woe_'+nb_name.split('.')[0]+'/'):
    os.makedirs('./woe_'+nb_name.split('.')[0]+'/')

In [ ]:
woe_dict_dir='./woe_'+nb_name.split('.')[0]+'/'

In [ ]:
def reduce_cardinality(df, field, threshold = 15):
    
    df[field]=df[field].fillna("missing_val")
    value_counts = df[field].astype(str).value_counts()
    all_categories = value_counts.index.tolist()
    keep_categories = (value_counts[value_counts > threshold]).index.tolist()
    remove_categories = list(set(all_categories) - set(keep_categories))
    reduce_card_dict = dict(zip(all_categories, all_categories))
    reduce_card_dict_2 = dict(zip(remove_categories, ['Other']*len(remove_categories)))
    reduce_card_dict.update(reduce_card_dict_2)

    card_dict_filename = '{0}{1}.csv'.format(woe_dict_dir, 'card_dict_feature_{0}'.format(field))
    card_dict_df = pd.DataFrame(list(reduce_card_dict.items()), columns = ['raw categories', 'mapped category'] )
    card_dict_df.to_csv(card_dict_filename, index = False)
    #print(card_dict_df['raw categories'], card_dict_df['mapped category'])
    #print(card_dict_filename)
    #print(card_dict_df)
    card_dict_df = card_dict_df.set_index('raw categories').to_dict()['mapped category']
    #card_dict_df = pd.read_csv(carmd_dict_filename).set_index('raw categories').to_dict()['mapped category']
    #card_dict_df = pd.read_csv(card_dict_filename).set_index('raw categories').to_dict()['mapped category']
    woe_field = "woe_"+field
    df[woe_field] = df[field].apply(lambda x: card_dict_df[x] if x in card_dict_df.keys() else np.nan) #used to be (str(x))
    
    return df
def woe_mapping(df, feature_column, target):
   
    epsilon_percent = 0.05
    groups = pd.crosstab(df[feature_column], df[target])
    #print(groups)
    categories = groups.columns.tolist()
    #print(categories)
    count_one = groups[categories[0]]
    mean_one = count_one/count_one.sum()
    
    try:
        count_two = groups[categories[1]]
        mean_two = count_two/count_two.sum()
    except:
        count_two = 0

    odds = mean_two/(mean_one + 0.001)
    log_odds = np.log(odds + epsilon_percent)
    log_odds.index = log_odds.index.astype(str)


    woe_mapping = dict(log_odds)
    woe_dict_filename = '{0}{1}.csv'.format(woe_dict_dir, 'woe_dict_feature_{0}'.format(feature_column))
    woe_dict_df = pd.DataFrame(list(woe_mapping.items()), columns = ['raw categories', 'woe'] )
    woe_dict_df.to_csv(woe_dict_filename, index = False)
    #woe_mapping = pd.read_csv(woe_dict_filename).set_index('raw categories').to_dict()['woe']
    woe_mapping = woe_dict_df.set_index('raw categories').to_dict()['woe']
    df[feature_column] = df[feature_column].apply(lambda x: woe_mapping[x] if x in woe_mapping.keys() else np.nan)
        
    return df

In [ ]:
train = train.copy()
for column in object_columns:
    print(column)
#     train = woe_mapping(reduce_cardinality(train, column), 'woe_'+column, 'delta_roi_model1')

### Find WOE value for validation and test data

In [ ]:
def card_assign(df, field, threshold = 15):
    card_dict_filename = '{0}{1}.csv'.format(woe_dict_dir, 'card_dict_feature_{0}'.format(field))
    card_dict_df = pd.read_csv(card_dict_filename).set_index('raw categories').to_dict()['mapped category']
    woe_field = "woe_"+field
    df[woe_field] = df[field].apply(lambda x: card_dict_df[x] if x in card_dict_df.keys() else np.nan) #used to be (str(x))
    return df

def woe_assign(df, feature_column, target):
    woe_dict_filename = '{0}{1}.csv'.format(woe_dict_dir, 'woe_dict_feature_{0}'.format(feature_column))
    woe_mapping = pd.read_csv(woe_dict_filename).set_index('raw categories').to_dict()['woe']
    df[feature_column] = df[feature_column].apply(lambda x: woe_mapping[x] if x in woe_mapping.keys() else np.nan)
    return df

In [ ]:
for column in object_columns:
    print(column)
#     train = woe_assign(card_assign(train, column), 'woe_'+column, 'delta_roi_model1')
    test = woe_assign(card_assign(test, column), 'woe_'+column, 'delta_roi_model1')
#     hold_df= woe_assign(card_assign(hold_df, column), 'woe_'+column, 'delta_roi_model1')
    oot_df= woe_assign(card_assign(oot_df, column), 'woe_'+column, 'delta_roi_model1')

In [ ]:
print(train.shape,test.shape,oot_df.shape,len(all_cols))

In [182]:
train.reset_index().to_feather("./"+woe_dict_dir+"train_df_model1.feather")
test.reset_index().to_feather("./"+woe_dict_dir+"test_df_model1.feather")
oot_df.reset_index().to_feather("./"+woe_dict_dir+"jan_df_model1.feather")